# read dimension data

In [1]:
import datetime
import numpy as np
import pandas as pd

In [2]:
from bs4 import BeautifulSoup
import requests
import json 
from datetime import datetime

In [3]:
import multiprocessing as mp

# Load data

In [4]:
dfCov=pd.read_json("dfpubl.json")

In [5]:
len(dfCov)

2891

# Scraping dimension archiv

In [6]:
## create a soup column
def getsoup0(x):
    return(BeautifulSoup(x,"lxml"))

In [7]:
## get html pages from dimension
def gethtml0(x):
    html=""
    url=x
    if url!=None:
        html = requests.get(url).content
    return(html)

In [ ]:
%%time
with mp.Pool(16) as pool:
    dfCov['htmldimension'] = pool.map(gethtml0, dfCov['Dimensions URL'])

In [196]:
%%time
#with mp.Pool(mp.cpu_count()) as pool:
with mp.Pool(16) as pool:
    dfCov["soup"]=pool.map(getsoup0,dfCov["htmldimension"])

CPU times: user 13.5 s, sys: 776 ms, total: 14.3 s
Wall time: 18.2 s


In [197]:
## create a dict column
def getdimdict0(x):
    datadoc=x.find("div")
    d=""
    if datadoc==None:
        attr=""
    else:
        attr=datadoc.get("data-doc")
        if attr:
            d=json.loads(attr)
    return(d)

In [198]:
%%time
dfCov["dimdict"]=dfCov["soup"].apply(getdimdict0)

CPU times: user 1.88 s, sys: 47.2 ms, total: 1.93 s
Wall time: 1.94 s


In [199]:
dfCov.head(3)

,index,Rank,Publication ID,DOI,PMID,PMCID,Title,Source title,Anthology title,Publisher,...,HRCS HC Categories,HRCS RAC Categories,ICRP Cancer Types,ICRP CSO Categories,Units of Assessment,Sustainable Development Goals,sourcefile,htmldimension,soup,dimdict
0,0,911,pub.1125763813,10.1016/j.ijantimicag.2020.105948,32201353.0,None,A review of the 2019 Novel Coronavirus (COVID-...,International Journal of Antimicrobial Agents,None,Elsevier,...,None,None,None,None,"A04 Psychology, Psychiatry and Neuroscience",None,../../dimensionxxl/Dimensions-Publication-2020...,"b'\n\n<!DOCTYPE html>\n<html lang=""en"">\n<head...","[html, [\n, [\n, OneTrust Cookies Consent Not...",{'title': 'A review of the 2019 Novel Coronavi...
1,1,891,pub.1125406007,10.1016/j.ijid.2020.03.003,32147538.0,PMC7129213,Recurrence of positive SARS-CoV-2 RNA in COVID...,International Journal of Infectious Diseases,None,Elsevier,...,Infection,None,None,None,A05 Biological Sciences,None,../../dimensionxxl/Dimensions-Publication-2020...,"b'\n\n<!DOCTYPE html>\n<html lang=""en"">\n<head...","[html, [\n, [\n, OneTrust Cookies Consent Not...",{'title': 'Recurrence of positive SARS-CoV-2 R...
2,2,887,pub.1124911473,10.1016/j.ijantimicag.2020.105924,32081636.0,PMC7127800,Severe acute respiratory syndrome coronavirus ...,International Journal of Antimicrobial Agents,None,Elsevier,...,Infection,None,None,None,A01 Clinical Medicine,None,../../dimensionxxl/Dimensions-Publication-2020...,"b'\n\n<!DOCTYPE html>\n<html lang=""en"">\n<head...","[html, [\n, [\n, OneTrust Cookies Consent Not...",{'title': 'Severe acute respiratory syndrome c...


In [200]:
now = datetime.now() # current date and time
date_time = now.strftime("%m_%d_%Y__%H_%M_%S")
print("date and time:",date_time)

date and time: 05_30_2020__14_48_01


In [201]:
dfCov.to_json("fullpubl.json")